## Mount Google Drive

In [22]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


## Imports

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [24]:
print(torch.__version__)
print(torchvision.__version__)

1.10.0+cu111
0.11.1+cu111


## Configs


In [25]:
torch.set_printoptions(linewidth=130)
torch.set_grad_enabled(True)

## Architecture

In [41]:
class MyVgg16(nn.Module):
  def __init__(self):
    super().__init__()

    # Define your layers as class attributes
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding='valid')
    self.conv1_2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding='valid')
    self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding='valid')
    self.conv2_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding='valid')
    self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding='valid')
    self.conv3_2 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding='valid')
    self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding='valid')
    self.conv4_2 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding='valid')

    self.fc1 = nn.Linear(in_features=512*7*7, out_features=4096)
    self.fc2 = nn.Linear(in_features=4096, out_features=4096)
    self.out = nn.Linear(in_features=4096, out_features=10)
  
  #implement network's forward pass in the forward() call. The __call__ will invoke the forward method.
  def forward(self, t):

    # First Block
    t = F.relu(self.conv1(t))
    t = F.relu(self.conv1_2(t))
    t = F.max_pool2d(t, kernel_size=3, stride=2)

    # Second Block
    t = F.relu(self.conv2(t))
    t = F.relu(self.conv2_2(t))
    t = F.max_pool2d(t, kernel_size=3, stride=2)

    # Third Block
    t = F.relu(self.conv3(t))
    t = F.relu(self.conv3_2(t))
    t = F.relu(self.conv3_2(t))
    t = F.max_pool2d(t, kernel_size=3, stride=2)

    # Fourth Block
    t = F.relu(self.conv4(t))
    t = F.relu(self.conv4_2(t))
    t = F.relu(self.conv4_2(t))
    t = F.max_pool2d(t, kernel_size=3, stride=2)

    # Flatten
    t = t.reshape((-1, 7*7*512))

    # FC 1
    t = F.relu(self.fc1(t))
    # FC 2
    t = F.relu(self.fc2(t))
    # Output
    t = self.out(t)

    return t



## Data

### FashionMNIST

In [36]:
train_set = torchvision.datasets.FashionMNIST(
    root = './data',
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Resize(224)])
)

In [37]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=8)

#### Sample the Data

In [38]:
images,labels = next(iter(train_loader))
print(labels)
print(images.shape)

tensor([9, 0, 0, 3, 0, 2, 7, 2])
torch.Size([8, 1, 224, 224])


## Train


In [42]:
#Instantiate the model.
model = MyVgg16()

# Give model's parameters to optimizer. Later, when the gradients are computer,
# they will be stored in each parameter, so the optimizer can perfofrm the step,
# by just having access to parameters.

optimizer = optim.Adam(model.parameters(), lr=1e-2)

In [ ]:
for epoch in range(10):
  
  epoch_loss = 0
  for batch in train_loader:
    images, labels = batch

    preds = model(images)
    batch_loss = F.cross_entropy(preds, labels)

    optimizer.zero_grad() # Because in successive iterations PyTorch will accumulate 
                          # gradient from previous steps.

    batch_loss.backward() # Calculare Gradients
    # Now we have gradients at each parameter
    # This can be viewed by : model.conv1.weight.grad tensor

    optimizer.step() # Update Weights
    print(batch_loss)
    epoch_loss += batch_loss

  print(epoch_loss)

tensor(2.2951, grad_fn=<NllLossBackward0>)
tensor(4.7011e+08, grad_fn=<NllLossBackward0>)
tensor(353.6143, grad_fn=<NllLossBackward0>)
tensor(24.7315, grad_fn=<NllLossBackward0>)
tensor(20438.3047, grad_fn=<NllLossBackward0>)
tensor(2639.2219, grad_fn=<NllLossBackward0>)
tensor(110.4575, grad_fn=<NllLossBackward0>)
